# Multimodal RAG with Amazon Bedrock Knowledge Base and AOSS

In today's data-driven world, the ability to effectively retrieve and generate relevant information from diverse sources is becoming increasingly crucial. Retrieval-Augmented Generation (RAG) models have emerged as a powerful approach to tackle this challenge, combining the strengths of retrieval systems and language models. In this blog post, we'll explore how to build a multimodal RAG solution using Amazon Bedrock Knowledge Base, Claude 3 (an AI assistant), and Amazon OpenSearch Serverless.

Amazon Bedrock Knowledge Base is a fully managed service that allows you to create and manage private knowledge bases from various data sources, including text, images, and videos. By leveraging Bedrock, we can create a centralized repository of information tailored to our specific needs.

Anthropics Claude 3, an advanced visual language model available on Amazon Bedrock, brings its powerful visual and natural language processing capabilities to the table. With its ability to understand and generate human-like text, Claude 3 can effectively process and synthesize information from the knowledge base.

Amazon OpenSearch Serverless, a fully managed service based on OpenSearch (a fork of Elasticsearch), provides a scalable and cost-effective solution for indexing and searching our knowledge base. Its serverless architecture ensures seamless scaling and eliminates the need for infrastructure management.

By combining these three components, we can create a multimodal RAG solution that can retrieve relevant information from our knowledge base, process it using Claude 3, and generate coherent and contextual responses. This approach has numerous applications, ranging from customer support and knowledge management to research and content creation.

In the following sections, we'll walk through the steps to set up and configure each component, integrate them into a cohesive solution, and demonstrate its capabilities with a practical example.

## Prepare environment

In [1]:
%pip install streamlit --upgrade
!pip install streamlit_jupyter --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.10
    Uninstalling jupyterlab_widgets-3.0.10:
      Successfully uninstalled jupyterlab_widgets-3.0.10
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbe

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
%load_ext autoreload
%autoreload 2

## Import needed packages

In [4]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm

In [5]:
sp = StreamlitPatcher()
sp.jupyter()  # register patcher with streamlit

## Start Streamlit UI

In [6]:
st.set_page_config(page_title="MM-RAG Demo",page_icon="🩺",layout="wide")
st.title("Multimodal Demo")

# Multimodal Demo

## Integrate with mmRAG

### ST sidebar

In [11]:
#@st.cache_data
#@st.cache_resource
st.title(':orange[Multimodal Config] :pencil2:')
option = st.selectbox('Choose Model',('anthropic.claude-3-haiku-20240307-v1:0', 
                                      'anthropic.claude-3-sonnet-20240229-v1:0', 
                                      'mistral.mistral-large-2402-v1:0'))

st.write("------- Default parameters ----------")
temperature = st.number_input("Temperature", min_value=0.0, max_value=1.0, value=0.1, step=0.05)
max_token = st.number_input("Maximum Output Token", min_value=0, value=1024, step=64)
top_p = st.number_input("Top_p: The cumulative probability cutoff for token selection", min_value=0.1, value=0.85)
top_k = st.number_input("Top_k: Sample from the k most likely next tokens at each step", min_value=1, value=40)
#candidate_count = st.number_input("Number of generated responses to return", min_value=1, value=1)
stop_sequences = st.text_input("The set of character sequences (up to 5) that will stop output generation", value="\n\n\n")

# :orange[Multimodal Config] :pencil2:

Dropdown(description='Choose Model', options=('anthropic.claude-3-haiku-20240307-v1:0', 'anthropic.claude-3-so…

------- Default parameters ----------

Textarea(value='\n\n\n', description='The set of character sequences (up to 5) that will stop output generatio…

### ST body

In [10]:
if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "I am your assistant. How can I help today?"}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

AttributeError: st.session_state has no attribute "messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization